In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20200613.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],Rank your movie choices. You don't have to do a full ranking. [Seventh Choice],Rank your movie choices. You don't have to do a full ranking. [Eighth Choice],Rank your movie choices. You don't have to do a full ranking. [Nineth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,6/3/2020 19:32:51,Caché,La Haine,Ivan's Childhood,Hearts of Darkness: A Filmmaker's Apocalypse,The Lion In Winter,Crash,Your Name,Outside the Law,NaN,No Theme/Free-for-all,Erotic Thriller,Filmmaker's First Feature,Martial Arts/Samurai,War
1,6/3/2020 19:40:37,Outside the Law,La Haine,Your Name,Hearts of Darkness: A Filmmaker's Apocalypse,Come and See,Caché,Ivan's Childhood,Crash,The Lion In Winter,Erotic Thriller,Martial Arts/Samurai,Filmmaker's First Feature,War,No Theme/Free-for-all
2,6/3/2020 19:50:36,Hearts of Darkness: A Filmmaker's Apocalypse,Your Name,La Haine,Ivan's Childhood,Come and See,Outside the Law,Caché,Crash,The Lion In Winter,Filmmaker's First Feature,Martial Arts/Samurai,War,No Theme/Free-for-all,Erotic Thriller
3,6/3/2020 20:29:44,Crash,Come and See,La Haine,The Lion In Winter,Ivan's Childhood,Outside the Law,Caché,Hearts of Darkness: A Filmmaker's Apocalypse,Your Name,Erotic Thriller,Filmmaker's First Feature,Martial Arts/Samurai,War,No Theme/Free-for-all
4,6/3/2020 22:25:38,Come and See,The Lion In Winter,Your Name,Hearts of Darkness: A Filmmaker's Apocalypse,Ivan's Childhood,Caché,Crash,La Haine,Outside the Law,Martial Arts/Samurai,War,Filmmaker's First Feature,Erotic Thriller,No Theme/Free-for-all
5,6/4/2020 7:55:47,Hearts of Darkness: A Filmmaker's Apocalypse,Your Name,Outside the Law,La Haine,Come and See,The Lion In Winter,Ivan's Childhood,Caché,Crash,Filmmaker's First Feature,War,Martial Arts/Samurai,No Theme/Free-for-all,Erotic Thriller
6,6/4/2020 9:29:10,Outside the Law,Hearts of Darkness: A Filmmaker's Apocalypse,La Haine,The Lion In Winter,NaN,NaN,NaN,NaN,NaN,Martial Arts/Samurai,Filmmaker's First Feature,War,Erotic Thriller,No Theme/Free-for-all
7,6/4/2020 10:29:04,Hearts of Darkness: A Filmmaker's Apocalypse,The Lion In Winter,Crash,Your Name,Outside the Law,NaN,NaN,NaN,NaN,No Theme/Free-for-all,Erotic Thriller,Filmmaker's First Feature,NaN,NaN
8,6/4/2020 11:51:10,Come and See,The Lion In Winter,Your Name,Outside the Law,La Haine,Ivan's Childhood,Hearts of Darkness: A Filmmaker's Apocalypse,Crash,Caché,Martial Arts/Samurai,War,Filmmaker's First Feature,Erotic Thriller,No Theme/Free-for-all
9,6/4/2020 15:45:40,Caché,Hearts of Darkness: A Filmmaker's Apocalypse,The Lion In Winter,La Haine,Outside the Law,Ivan's Childhood,Crash,Come and See,Your Name,War,Filmmaker's First Feature,Erotic Thriller,No Theme/Free-for-all,Martial Arts/Samurai


In [8]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [9]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Outside the Law', 'La Haine', 'Your Name',
       "Hearts of Darkness: A Filmmaker's Apocalypse", 'Come and See',
       'Caché', "Ivan's Childhood", 'Crash', 'The Lion In Winter'],
      dtype=object)

In [10]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [11]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [12]:
submission_dict

{'Outside the Law': <Candidate('Outside the Law')>,
 'La Haine': <Candidate('La Haine')>,
 'Your Name': <Candidate('Your Name')>,
 "Hearts of Darkness: A Filmmaker's Apocalypse": <Candidate('Hearts of Darkness: A Filmmaker's Apocalypse')>,
 'Come and See': <Candidate('Come and See')>,
 'Caché': <Candidate('Caché')>,
 "Ivan's Childhood": <Candidate('Ivan's Childhood')>,
 'Crash': <Candidate('Crash')>,
 'The Lion In Winter': <Candidate('The Lion In Winter')>}

In [13]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [14]:
ballots

[<Ballot(Caché, La Haine, Ivan's Childhood, Hearts of Darkness: A Filmmaker's Apocalypse, The Lion In Winter, Crash, Your Name, Outside the Law)>,
 <Ballot(Outside the Law, La Haine, Your Name, Hearts of Darkness: A Filmmaker's Apocalypse, Come and See, Caché, Ivan's Childhood, Crash, The Lion In Winter)>,
 <Ballot(Hearts of Darkness: A Filmmaker's Apocalypse, Your Name, La Haine, Ivan's Childhood, Come and See, Outside the Law, Caché, Crash, The Lion In Winter)>,
 <Ballot(Crash, Come and See, La Haine, The Lion In Winter, Ivan's Childhood, Outside the Law, Caché, Hearts of Darkness: A Filmmaker's Apocalypse, Your Name)>,
 <Ballot(Come and See, The Lion In Winter, Your Name, Hearts of Darkness: A Filmmaker's Apocalypse, Ivan's Childhood, Caché, Crash, La Haine, Outside the Law)>,
 <Ballot(Hearts of Darkness: A Filmmaker's Apocalypse, Your Name, Outside the Law, La Haine, Come and See, The Lion In Winter, Ivan's Childhood, Caché, Crash)>,
 <Ballot(Outside the Law, Hearts of Darkness: A 

In [15]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [16]:
print(election_result)

ROUND 1
Candidate                                       Votes  Status
--------------------------------------------  -------  --------
Hearts of Darkness: A Filmmaker's Apocalypse        3  Hopeful
Come and See                                        3  Hopeful
Outside the Law                                     2  Hopeful
Caché                                               2  Hopeful
La Haine                                            1  Rejected
Crash                                               1  Rejected
Your Name                                           0  Rejected
The Lion In Winter                                  0  Rejected
Ivan's Childhood                                    0  Rejected

ROUND 2
Candidate                                       Votes  Status
--------------------------------------------  -------  --------
Come and See                                        4  Hopeful
Hearts of Darkness: A Filmmaker's Apocalypse        3  Hopeful
Caché                            

In [18]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                    Votes  Status
-------------------------  -------  --------
Erotic Thriller                  3  Hopeful
Martial Arts/Samurai             3  Hopeful
No Theme/Free-for-all            3  Hopeful
Filmmaker's First Feature        2  Rejected
War                              1  Rejected

ROUND 2
Candidate                    Votes  Status
-------------------------  -------  --------
Martial Arts/Samurai             5  Hopeful
Erotic Thriller                  4  Hopeful
No Theme/Free-for-all            3  Rejected
Filmmaker's First Feature        0  Rejected
War                              0  Rejected

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
Erotic Thriller                  7  Elected
Martial Arts/Samurai             5  Rejected
No Theme/Free-for-all            0  Rejected
Filmmaker's First Feature        0  Rejected
War                              0  Rejected

